In [6]:
# Import Necessary python libraries like numpy, pandas, os, and sys
import numpy as np
import pandas as pd
import os
import sys

In [7]:
# Get the current working directory
cwd = os.getcwd()
print(cwd)

/Users/deancochran/Dev/cadence/ml


In [8]:
def calc_cIF(NP, FTP):
    return NP / FTP

def effort_power_output(S, NP, INTENSITY_FACTOR):
    return S * NP * INTENSITY_FACTOR


In [13]:
# Inside the data/raw folder, create a seelected folder called zwift_workouts_all_collections_ordered_Nov20
# read all files in all folders and subfolders with the .xml or .zwo extension
# store each file in the data/prepped folder
from bs4 import BeautifulSoup   

# Make a new dataframe
df = pd.DataFrame(columns=['name','description','sportType', 'intervals'])
index=0
for root, dirs, files in os.walk('data/'):
    for file in files:
        pass
        if file.endswith(".xml") or file.endswith(".zwo"):
            path = os.path.join(root, file)
            print(path)
            file = open(path,'r') 
            contents = file.read()
            soup = BeautifulSoup(contents,'xml')
            name=soup.find('name')
            description=soup.find('description')
            sportType=soup.find('sportType')
            # Find the Workout Tag
            workout = soup.find('workout')
            intervals = []
            for interval in workout.children:
                match interval.name:
                    case "Cooldown":
                        #  Duration="360" PowerLow="0.5"
                        # Read the Duration and PowerLow attribute values, convert them to numbers
                        duration = float(interval['Duration'])
                        try:
                            power_low = float(interval['PowerLow'])
                        except:
                            power_low = float(interval['Power'])
                        
                        intervals.append({'duration': duration, 'intensity': power_low})
                        pass
                    case "cooldown":
                        # Duration="420" PowerLow="0.6" PowerHigh="0.3">
                        duration = float(interval['Duration'])
                        power_low = float(interval['PowerLow'])
                        power_high = float(interval['PowerHigh'])
                        intervals.append({'duration': duration, 'intensity': power_low})
                        pass
                    case "FreeRide":
                        # FreeRide Duration="2700"
                        duration = float(interval['Duration'])
                        intervals.append({'duration': duration, 'intensity': .4})
                        pass
                    case "Freeride":
                        # FreeRide Duration="2700"
                        duration = float(interval['Duration'])
                        intervals.append({'duration': duration, 'intensity': .4})
                        pass
                    case "IntervalsT":
                        #  Repeat="5" OnDuration="240" OffDuration="120" OffPower="0.52" OnPower="0.72" 
                        repeat = int(interval['Repeat'])
                        on_duration = float(interval['OnDuration'])
                        off_duration = float(interval['OffDuration'])
                        try:
                            off_power = float(interval['OffPower'])
                            on_power = float(interval['OnPower'])
                        except:
                            try:
                                off_power = float(interval['PowerOffLow'])
                                on_power = float(interval['PowerOnLow'])
                            except:
                                match interval["PowerOnZone"]:
                                    case "1":
                                        on_power = .2
                                        break
                                    case "2":
                                        on_power = .4
                                        break
                                    case "3":
                                        on_power = .5
                                        break
                                    case "4":
                                        on_power = .7
                                        break
                                    case "5":
                                        on_power = 1
                                        break
                                    case "6":
                                        on_power = 1.2
                                        break
                                    case "7":
                                        on_power = 1.3
                                        break
                                    case _:
                                        on_power = .5

                                match interval["PowerOffZone"]:
                                    case "1":
                                        off_power = .2
                                        break
                                    case "2":
                                        off_power = .4
                                        break
                                    case "3":
                                        off_power = .5
                                        break
                                    case "4":
                                        off_power = .7
                                        break
                                    case "5":
                                        off_power = 1
                                        break
                                    case "6":
                                        off_power = 1.2
                                        break
                                    case "7":
                                        off_power = 1.3
                                        break
                                    case _:
                                        
                                        off_power = .5

                            
                        
                        
                        for i in range(repeat):
                            intervals.append({'duration': on_duration, 'intensity': on_power})
                            intervals.append({'duration': off_duration, 'intensity': off_power})
                        pass
                    case "MaxEffort":
                        #  Duration="8"
                        duration = float(interval['Duration'])
                        intervals.append({'duration': duration, 'intensity': 2.0})
                        pass
                    case "Ramp":
                        duration = float(interval['Duration'])
                        try:
                            power_low = interval['Power']
                        except:
                            power_low = interval['PowerLow']
                        intervals.append({'duration': duration, 'intensity': power_low})
                        pass
                    case "RestDay":
                        pass
                    case "SolidState":
                        # Duration="300" Power="0.3">
                        duration = float(interval['Duration'])
                        power = float(interval['Power'])
                        intervals.append({'duration': duration, 'intensity': power})
                        pass
                    case "SteadyState":
                        #  Duration="600" PowerLow="0.5"
                        duration = float(interval['Duration'])
                        try:
                            try:
                                power_low = interval['Power']
                            except:
                                power_low = interval['PowerLow']
                        except:
                            match interval["Zone"]:
                                case "1":
                                    power_low = .2
                                    break
                                case "2":
                                    power_low = .4
                                    break
                                case "3":
                                    power_low = .5
                                    break
                                case "4":
                                    power_low = .7
                                    break
                                case "5":
                                    power_low = 1
                                    break
                                case "6":
                                    power_low = 1.2
                                    break
                                case "7":
                                    power_low = 1.3
                                    break
                                case _:
                                    power_low = .5
                                    break

                        
                        intervals.append({'duration': duration, 'intensity': power_low})
                        pass
                    case "Warmup":
                        #  Duration="780" PowerLow="0.5" PowerHigh="0.9"
                        duration = float(interval['Duration'])
                        
                        try:
                            try:
                                power_low = interval['Power']
                            except:
                                power_low = interval['PowerLow']
                        except:
                            match interval["Zone"]:
                                case "1":
                                    power_low = .2
                                    break
                                case "2":
                                    power_low = .4
                                    break
                                case "3":
                                    power_low = .5
                                    break
                                case "4":
                                    power_low = .7
                                    break
                                case "5":
                                    power_low = 1
                                    break
                                case "6":
                                    power_low = 1.2
                                    break
                                case "7":
                                    power_low = 1.3
                                    break
                                case _:
                                    power_low = .5
                                    break
                            
                        intervals.append({'duration': duration, 'intensity': power_low})
                        pass
                    case "textevent":
                        pass
                    case _:
                        pass

            row = {'name':name, 'description':description, 'sportType':sportType, 'intervals':intervals}
            df = df._append(row, ignore_index = True)


            file.close()

    index=index+1



data/raw/zwift_workouts_all_collections_ordered_Nov20/Fun_is_Fast_2021/AvdB_Over_Under.zwo
data/raw/zwift_workouts_all_collections_ordered_Nov20/Fun_is_Fast_2021/MvdP_Anaerobic.zwo
data/raw/zwift_workouts_all_collections_ordered_Nov20/Fun_is_Fast_2021/Geraint_Thomas_Climb.zwo
data/raw/zwift_workouts_all_collections_ordered_Nov20/Zwift_Academy_Tri_2021/Bike/3_Half_Distance_Development.zwo
data/raw/zwift_workouts_all_collections_ordered_Nov20/Zwift_Academy_Tri_2021/Bike/5_LongVO2max.zwo
data/raw/zwift_workouts_all_collections_ordered_Nov20/Zwift_Academy_Tri_2021/Bike/4_ShortVO2_Intervals.zwo
data/raw/zwift_workouts_all_collections_ordered_Nov20/Zwift_Academy_Tri_2021/Bike/1_Full_Distance_Specific.zwo
data/raw/zwift_workouts_all_collections_ordered_Nov20/Zwift_Academy_Tri_2021/Bike/2_Strength_Endurance_Threshold_Combo.zwo
data/raw/zwift_workouts_all_collections_ordered_Nov20/Zwift_Academy_Tri_2021/Bike/ZA_Tri_Bike_2021_Orientation.zwo
data/raw/zwift_workouts_all_collections_ordered_Nov20/

data/raw/zwift_workouts_all_collections_ordered_Nov20/Time_Crunched/Vault.zwo
data/raw/zwift_workouts_all_collections_ordered_Nov20/Time_Crunched/Knife_edge.zwo
data/raw/zwift_workouts_all_collections_ordered_Nov20/Time_Crunched/Lavendar_Unicorn.zwo
data/raw/zwift_workouts_all_collections_ordered_Nov20/Time_Crunched/Expand.zwo
data/raw/zwift_workouts_all_collections_ordered_Nov20/British_Cycling_eRacing_Championship/Week_2/2040.zwo
data/raw/zwift_workouts_all_collections_ordered_Nov20/British_Cycling_eRacing_Championship/Week_2/5_min_VO2.zwo
data/raw/zwift_workouts_all_collections_ordered_Nov20/British_Cycling_eRacing_Championship/Week_2/Free_Ride_Week_2.zwo
data/raw/zwift_workouts_all_collections_ordered_Nov20/British_Cycling_eRacing_Championship/Week_2/2X20.zwo
data/raw/zwift_workouts_all_collections_ordered_Nov20/British_Cycling_eRacing_Championship/Week_5/Double_Warm_Up.zwo
data/raw/zwift_workouts_all_collections_ordered_Nov20/British_Cycling_eRacing_Championship/Week_5/Leg_Speed.z

In [14]:
df

,name,description,sportType,intervals
0,[Fun is Staying Cool],[Anna van der Breggen is arguably one of the m...,[bike],"[{'duration': 500.0, 'intensity': '0.5'}, {'du..."
1,[Fun is Going Full Gas],"[Since he was a Junior, Mathieu van der Poel (...",[bike],"[{'duration': 600.0, 'intensity': '.5'}, {'dur..."
2,[Fun is Flying Uphill],"[Steady tempo efforts are great, but by incorp...",[bike],"[{'duration': 480.0, 'intensity': '.5'}, {'dur..."
3,[70.3 Development],[Looking for some brick work? You can pair tod...,[bike],"[{'duration': 300.0, 'intensity': '0.35'}, {'d..."
4,[Long VO2 max],[By now you should have already been through s...,[bike],"[{'duration': 240.0, 'intensity': '0.35'}, {'d..."
...,...,...,...,...
1488,[Week 4.1 - Wave Rider],"[They call it the ""Wave Rider"" because the wor...",[bike],"[{'duration': 480.0, 'intensity': '0.5'}, {'du..."
1489,[Week 2.1 - Twenty Torch],"[You can do anything for 20min, right!? Good! ...",[bike],"[{'duration': 180.0, 'intensity': '.65'}, {'du..."
1490,[Week 2.2 - Calorie Crush],[Back at it with another super fun workout! We...,[bike],"[{'duration': 180.0, 'intensity': '.65'}, {'du..."
1491,[Week 1.2 - Instant Inferno],[Let's keep the consistency going with a quick...,[bike],"[{'duration': 60.0, 'intensity': '.65'}, {'dur..."


In [15]:
df.to_csv('data/prepped/zwift_workouts_all_collections_ordered_Nov20.csv', index=False)